# Урок 5. Задача оттока: варианты постановки, возможные способы решения

**Задание**


1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями:
- бустинг
- логистическая регрессия
2. Отобрать лучшую модель по метрикам (какая по вашему мнению здесь наиболее подходящая ML-метрика)
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2:
- 1 доллар на удержание
- 2 доллара - с каждого правильно классифицированного (True Positive)

4. *Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. *Еще раз провести оценку экономической эффективности

In [1]:
# !./bash/hw5.sh

In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import roc_auc_score, precision_recall_curve, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

from lib.transformers import *

In [3]:
%matplotlib inline

In [4]:
df = pd.read_csv("./data/churn_data.csv")
df.sample(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
8507,8508,15771749,Duncan,653,Germany,Female,38,5,114268.22,2,1,1,89524.83,0
3578,3579,15711618,Chang,704,Germany,Female,39,1,124640.51,1,1,0,116511.12,1
9818,9819,15619699,Yeh,558,France,Male,31,7,0.00,1,1,0,198269.08,0
8080,8081,15668775,Pendred,757,France,Male,47,3,130747.10,1,1,0,143829.54,0
7023,7024,15605791,Li,524,Germany,Male,29,9,144287.60,2,1,0,32063.30,0


In [5]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [7]:
params = {
    "continuos_cols" : ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary'],
    "cat_cols" : ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
}

In [8]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [9]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
        ('selector', FeatureSelector(column=cat_col)),
        ('ohe', OHEEncoder(key=cat_col))
    ])

    final_transformers.append((cat_col, cat_transformer))

for cont_col in continuous_columns:
    cont_transformer = Pipeline([
        ('selector', NumberSelector(key=cont_col)),
        ('scaler', StandardScaler())
    ])

    final_transformers.append((cont_col, cont_transformer))


feats = FeatureUnion(final_transformers)

In [10]:
model_lr = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression(random_state=42)),
])


# обучим пайплайн на всем тренировочном датасете
model_lr.fit(X_train, y_train)

preds = model_lr.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.28952195521698365, F-Score=0.510, Precision=0.462, Recall=0.568


In [11]:
metrics_df = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC


In [12]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.7720774921330664

In [13]:
metrics_df = metrics_df.append({
    'model': type(model_lr['classifier']).__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

/var/folders/4m/q56tywrn4y17g19rm7bs5xxh0000gp/T/ipykernel_15498/2418686002.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,LogisticRegression,0.289522,0.5097,0.4624,0.56778,0.772077


In [14]:
model_gb = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier(random_state=42)),
])


# обучим пайплайн на всем тренировочном датасете
model_gb.fit(X_train, y_train)

preds = model_gb.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.4085078904556646, F-Score=0.646, Precision=0.704, Recall=0.597


In [15]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.8757458662211781

In [16]:
metrics_df = metrics_df.append({
    'model': type(model_gb['classifier']).__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

/var/folders/4m/q56tywrn4y17g19rm7bs5xxh0000gp/T/ipykernel_15498/2856801916.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


In [17]:
metrics_df.sort_values('F-Score')

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,LogisticRegression,0.289522,0.5097,0.4624,0.56778,0.772077
1,GradientBoostingClassifier,0.408508,0.646121,0.703704,0.59725,0.875746


In [18]:
preds = model_gb.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

cnf_matrix = confusion_matrix(y_test, preds > thresholds[ix])

In [19]:
TN = cnf_matrix[0][0]
FP = cnf_matrix[0][1]
FN = cnf_matrix[1][0]
TP = cnf_matrix[1][1]


retain_sum = (FP + TP) * 1
income = TP * 2

income - retain_sum

175

In [20]:
params = {
    'classifier__max_features': [0.3, 0.5, 0.7],
    'classifier__min_samples_leaf': [1, 15, 30, 50],
    'classifier__n_estimators': [50, 100, 150, 300]
}

In [21]:
grid = GridSearchCV(model_gb,
                    param_grid=params,
                    cv=5,
                    scoring='recall',
                    refit=False)

search = grid.fit(X_train, y_train)
search.best_params_

{'classifier__max_features': 0.3,
 'classifier__min_samples_leaf': 1,
 'classifier__n_estimators': 300}

In [22]:
model_gb_tuned = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier(n_estimators=300,
                                              min_samples_leaf=1,
                                              max_features=0.7,
                                              random_state=42)),
])


# обучим пайплайн на всем тренировочном датасете
model_gb_tuned.fit(X_train, y_train)

preds = model_gb_tuned.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.4333908349794679, F-Score=0.645, Precision=0.700, Recall=0.597


In [23]:
preds = model_gb_tuned.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

cnf_matrix = confusion_matrix(y_test, preds > thresholds[ix])

In [24]:
TN = cnf_matrix[0][0]
FP = cnf_matrix[0][1]
FN = cnf_matrix[1][0]
TP = cnf_matrix[1][1]


retain_sum = (FP + TP) * 1
income = TP * 2

income - retain_sum

173